In [1]:
import pandas as pd
import holidays
import calendar
import numpy as np
from datetime import date, datetime
import locale
from datetime import timedelta

# Cambiar el idioma del sistema a español
locale.setlocale(locale.LC_ALL, 'es_ES.UTF-8')

'es_ES.UTF-8'

In [2]:
meses = {'01': 'Ene', '02': 'Feb', '03': 'Mar', '04': 'Abr', \
       '05': 'May', '06': 'Jun', '07': 'Jul', '08': 'Ago', \
       '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dic'}

meses_int = {int(k): v for k, v in meses.items()}

global meses_int

In [3]:

# get class day without hollidays
def dias_clase(yy, month, selections=[True, False, True, False, True, False, False]):
    """
    Return class day without holliday day or activities day
    """    
    cal = np.array(calendar.monthcalendar(yy, month))
    # Esta es la funcion a cambiar dependiendo del dia de la semana    
    dias_clase_ = cal[:,selections].flatten()        
    tf = np.isin(dias_clase_, 0)
    return dias_clase_[~tf]

def get_holliday(yy,month):
    # get holliday day from month 
    festivos = holidays.country_holidays(country="CO", years=yy)
    f = np.array(list(map(lambda x: x.day if \
                          (x.month == month) else None, festivos)))
    dias_f = f[f!=None]
    return dias_f

def remove_day(dclase,day_to_remove):
    """
    Remove class days
    """
    tf = np.isin(dclase, day_to_remove)
    return dclase[~tf]

def sesion_day_per_month(day_without_hollidays,month, dicc={}):
    try: 
        remo = np.array(dicc[month])
        q = remove_day(day_without_hollidays,remo)
       # print(q)
        return q
    except:
        #print(day_without_hollidays)
        return day_without_hollidays  

    
def return_df_with_month_day(total_day):
    dates = []
    for key in total_day:
        for day in total_day[key]:        
            dates.append(f"{meses_int[key]} {day:02d}")
    df = pd.DataFrame({"Mes_dia":dates[0:]})
    return df   

def class_day(days: str):
    
    dicc = {"L":False,"M":False,"W":False,\
          "J":False,"V":False,"S":False,"D":False}
    
    for key in days :
        dicc[key] = True
    return list(dicc.values())


def append_week(df:pd.core.frame.DataFrame):
    
    df["Fecha"] = pd.to_datetime(list(map(lambda x: x+" 2024",\
                                          df['Mes_dia'])),format="%b %d %Y")
    df = df.set_index(["Fecha"])
    week_number = np.array(list(map(lambda x: x.isocalendar()[1],df.index)))
    df["Semana"] = week_number + 1 - week_number[0]
    return df

def main_day_sesion(yy:str,begin_month, begin_day,\
                    dicc = {3:[1,4,5]}, \
                    selections=[True, False, True, False, True, False, False]):
    """

    dicc = Definir los días en lo que no se da clase, por cualquier motivo
           por ejemplo examenes de admision y los primeros dias de semana santa           
    
    key = month, value=list
    Example: dicc={4:[1,2,3,4]}
    dias de clase
    selections = [LMWJVSD]
    """
    
    total_day = {}    
    for i in range(begin_month, begin_month+5):
            
        d = dias_clase(yy,i,selections)
        h = get_holliday(yy,i)
        day_ = remove_day(d,h)
        # Remove aditional day in dicc
        sesion_day = sesion_day_per_month(day_, i, dicc)
        total_day[i] = sesion_day
        
        if(i == begin_month):            
            tf = total_day[i] >= begin_day
            total_day[i] = total_day[i][tf]          
        # aca puede incluirse el dia final con un condicional similar
    df = return_df_with_month_day(total_day)
    
    df = append_week(df)
    df.reset_index(inplace=True)
    df.index += 1
    df.reset_index(names="Sesion", inplace=True)

    

    return df


In [4]:

def generate_link(url, repo, branch, subpath=""):
    pull_git = "/hub/user-redirect/git-pull?repo="
    branch_ = f"&branch={branch}"
    subPath_ = f"&subPath={subpath}"
    app="&app=lab"

    if(len(subpath))!=0:
      url_valida=f"{url}{pull_git}{repo}{branch_}{subPath_}"
    else:
      url_valida=f"{url}{pull_git}{repo}{branch_}{app}"
    return url_valida


def load_paths(url, repo, branch):
    df_ = pd.read_excel("Libro1.xlsx", skiprows = 2)
    df_ = df_.fillna(0)
    columns = ["Name_sesion_es", "Lab","Hw", "Project", "Exam" ,"Ntipo", "Name_lab_es", \
           "Name_project_es","Sesion","Name_book","Book_url_values",\
               "Presentacion", "Video", "Demostracion"]
    df = df_[columns].copy()
    # Split data frame  
    df["Seccion_lecturas"]= 0
    for i,row in enumerate(df.Name_book):
        if isinstance(row, str):
            df.loc[i,"Seccion_lecturas"] = row.split()[0][0:-1]


    df["Tipo_lab"] = list(map(lambda x: f"lab/{x.lower()}/{x.lower()}.ipynb" \
                                  if isinstance(x, str) else None, df["Lab"]))
    for i , row in enumerate(df.Tipo_lab):
        if isinstance(row, str):
            df.loc[i,"Tipo_lab" ] = generate_link(url, repo, branch,row)



    df["Tipo_hw"] = list(map(lambda x: f"hw/{x.lower()}/{x.lower()}.ipynb" \
                                  if isinstance(x, str) else None, df["Hw"]))
    for i , row in enumerate(df.Tipo_hw):
        if isinstance(row, str):
            df.loc[i,"Tipo_hw" ] = generate_link(url, repo, branch,row)




    df["Tipo_exam"] = list(map(lambda x: f"exam/{x.lower()}/{x.lower()}.ipynb" \
                                  if isinstance(x, str) else None, df["Exam"]))
    for i , row in enumerate(df.Tipo_exam):
        if isinstance(row, str):
            df.loc[i,"Tipo_exam" ] = generate_link(url, repo, branch,row)


    df["Tipo_project"] = list(map(lambda x: f"project/{x.lower()}/{x.lower()}.ipynb" \
                                  if isinstance(x, str) else None, df["Project"]))
    for i , row in enumerate(df.Tipo_project):
        if isinstance(row, str):
            df.loc[i,"Tipo_project" ] = generate_link(url, repo, branch,row)

    df.to_csv("new_data_base.csv")
    return df

In [6]:
url = "http://localhost:8000"
repo= "https://github.com/hernansalinas/FundCienDatosMaterial_year_semestre_dev"
branch = "main" # La rama del repositorio de Git



Sesion_day = main_day_sesion(yy = 2024, begin_month = 8, begin_day = 10,\
                            selections = class_day("MJ"))
df = load_paths(url, repo, branch)
df.index += 1

/var/folders/6s/h5hh29jn1dqd4r627s043vy40000gn/T/ipykernel_66008/305337573.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i,"Seccion_lecturas"] = row.split()[0][0:-1]


In [21]:
a=Sesion_day.iloc[8:37].reset_index()
a["Semana"]=a["Semana"]-4
a.to_csv("hola.csv")

In [22]:
a.to_excel("programacion.xlsx")

In [6]:
#prog = Sesion_day.merge(df, left_index=True, right_index=True)
cols=[ 'Sesion', 'Name_sesion_es', 'Lab', 'Hw', 'Project',
       'Exam', 'Ntipo', 'Name_lab_es', 'Name_project_es' ,
       'Name_book', 'Book_url_values', 'Seccion_lecturas', 'Tipo_lab',
       'Tipo_hw', 'Tipo_exam', 'Tipo_project', \
      "Presentacion", "Video","Demostracion"]

#df["Presentacion"] = None
#df["Video"] = None
#df["Demostracion"]=None
df_ = df[cols]

In [7]:
def dicc_with_urls(url_presentacion, url_demostracion, url_video):
    """
    Revisa si la columna tiene valor en presentacion, demostracion o video
    """
    v=tf_url_presentacion,  tf_url_demostracion, tf_url_video = \
    [url_presentacion==0.0, url_demostracion==0.0, url_video==0.0 ]
    dicc = {}
    for i,tf in enumerate( v):
        if(i==0 and tf==False):
            dicc["Presentacion"] = f"({url_presentacion}){format_3} "
        elif(i==1 and tf==False):
            dicc["Demostracion"] = f"({url_demostracion}){format_3} "
        elif(i==2 and tf==False):
            dicc["Video"] = f"({url_video}){format_3} "

        else:
            ""
    return dicc


def dicc_lab_hw_project_urls(Lab_url, Hw_url, Project_url,Tipo_exam_url,clave, names,f_actual):
    """
    Revisa si la columna tiene valor en presentacion, demostracion o video
    """
    v = tf_Lab_url,  tf_Hw_url, tf_Project_url,tf_Tipo_exam_url = \
    [Lab_url==None, Hw_url ==None, Project_url==None ,Tipo_exam_url==None]
    dicc = {}
    
    x=f_actual.iloc[0]
    
    
    for i,tf in enumerate( v):
        if(i==0 and tf==False):
            color = format_4("green")            
            x = x+timedelta(days=7)
            fecha = x.strftime("%b %d").capitalize() # Formatear la fecha
            dicc[f"{clave[0]}: {names[0]}"] = f"({Lab_url}){color}<br> `(Entrega {fecha})`{{: .label .label-yellow}} "
            x=f_actual.iloc[0]

        elif(i==1 and tf==False):
            color = format_4("blue")                        
            x = x+timedelta(days = 5)
            fecha = x.strftime("%b %d").capitalize() # Formatear la fecha

            dicc[f"Tarea {clave[1][2:4]}"] = f"({Hw_url}){color} <br>`(Entrega {fecha})`{{: .label .label-yellow}} "
            x=f_actual.iloc[0]

            
        elif(i==2 and tf==False):
            color = format_4("purple")            
            x = x + timedelta(days = 10)
            fecha = x.strftime("%b %d").capitalize() # Formatear la fecha


            dicc[f"{clave[2]}: {names[2]}"] = f"({Project_url}){color} <br> `(Entrega {fecha})`{{: .label .label-yellow}} "
            x=f_actual.iloc[0]

        elif(i==3 and tf == False):
            color = format_4("red")            
            dicc[f"{clave[3]}"] = f"({Tipo_exam_url}){color} "
            x=f_actual.iloc[0]

        else:
            ""
    return dicc

In [8]:
format_1 = "{: .fw-700 .fs-5}"
format_2 = "{: .fw-700 .fs-3}"
format_3 = "{: .btn .float-left}"
format_4 = lambda x: f"{{: .label .label-{x} .float-left  }}"


head = (f""" <div class=\"code-example\" markdown=\"1\">
\
""")


Nsesion = 2

df_[df_.Sesion == Nsesion]
# day Sesion
MD = Sesion_day[Sesion_day.Sesion == Nsesion].Mes_dia.values[0]

#name sesion
NS = df_[df_.Sesion == Nsesion].Name_sesion_es.values[0]

#reading
z = []
for reading in df_[df_.Sesion == Nsesion].itertuples():
    z.append(f"[{reading.Seccion_lecturas}]({reading.Book_url_values})")

    

url_presentacion = df_[df_.Sesion == Nsesion].Presentacion.values[0]
url_demostracion = df_[df_.Sesion == Nsesion].Demostracion.values[0]
url_video = df_[df_.Sesion == Nsesion].Video.values[0]



b = (f"""{head}
|  Fecha   | Sesion          | 
|:---------|:--------------- |
| {MD}     | `{Nsesion}`{format_1} `{NS}`{format_1} <br><br>""")

In [9]:
dicc = dicc_with_urls(url_presentacion, url_demostracion, url_video)
reading = f" _`Lecturas:`{format_2}_"+"--".join(z)
c = b + reading + " <br><br>"
# Contruccion de string para visual estudio code
for a,b in dicc.items():
    c = c + f"[{a}]" + b 
    
c=c+"<br><br>"

In [10]:
Lab_url = df_[df_.Sesion == Nsesion].Tipo_lab.values[0]
Hw_url = df_[df_.Sesion == Nsesion].Tipo_hw.values[0]
Project_url = df_[df_.Sesion == Nsesion].Tipo_project.values[0]
Tipo_exam_url = df_[df_.Sesion == Nsesion].Tipo_exam.values[0]

In [11]:
L = df_[df_.Sesion == Nsesion].Lab.values[0] 


H = df_[df_.Sesion == Nsesion].Hw.values[0] 
P = df_[df_.Sesion == Nsesion].Project.values[0]  
W = df_[df_.Sesion == Nsesion].Exam.values[0]
clave = [L, H, P, W]

name_lab = df_[df_.Sesion == Nsesion].Name_lab_es.values[0]
name_proj = df_[df_.Sesion == Nsesion].Name_project_es.values[0]
names=[name_lab, None, name_proj, None]


f_actual = Sesion_day[Sesion_day.Sesion == Nsesion].Fecha+timedelta(days=7)

dicc = dicc_lab_hw_project_urls(Lab_url, Hw_url, Project_url,Tipo_exam_url, clave, names, f_actual)


In [12]:
# Contruccion de string para visual estudio code
for a,b in dicc.items():
    c = c + f"[{a}]" + b + "<br><br>"

In [13]:
archivo = open("ejemplo.txt", "w")
archivo.write(c)
# Cierra el archivo
archivo.close()

In [14]:
url_presentacion = df_[df_.Sesion == Nsesion].Presentacion.values[0]

# Generalización 

In [15]:
def get_html_week(df_,Sesion_day,head, Nsesion = 2):

    format_1 = "{: .fw-700 .fs-5}"
    format_2 = "{: .fw-700 .fs-3}"
    format_3 = "{: .btn .float-left}"
    format_4 = lambda x: f"{{: .label .label-{x} .float-left  }}"

    df_[df_.Sesion == Nsesion]
    # day Sesion
    MD = Sesion_day[Sesion_day.Sesion == Nsesion].Mes_dia.values[0]

    #name sesion
    NS = df_[df_.Sesion == Nsesion].Name_sesion_es.values[0]

    #reading
    z = []
    for reading in df_[df_.Sesion == Nsesion].itertuples():
        z.append(f"[{reading.Seccion_lecturas}]({reading.Book_url_values})")



    url_presentacion = df_[df_.Sesion == Nsesion].Presentacion.values[0]
    url_demostracion = df_[df_.Sesion == Nsesion].Demostracion.values[0]
    url_video = df_[df_.Sesion == Nsesion].Video.values[0]
   # print(Nsesion%3)
    if(Nsesion==1):
        b =(f"""{head}\

|  Fecha   | Sesion          |\

|:---------|:--------------- |\

| {MD}     | `{Nsesion}`{format_1} `{NS}`{format_1} <br><br>""")

    elif( (Nsesion-1)%3==0):
#        print("hlasdjflajdflakjdsflk")
        b =(f"""|  Fecha   | Sesion          |\

|:---------|:--------------- |\

| {MD}     | `{Nsesion}`{format_1} `{NS}`{format_1} <br><br>""")

    elif(Nsesion>1 and (Nsesion-1)%3!=0):
        b =(f"""
| {MD}     | `{Nsesion}`{format_1} `{NS}`{format_1} <br><br>""")
        
        
        
    else:
        print("hola mundop")
    
        
        
    dicc = dicc_with_urls(url_presentacion, url_demostracion, url_video)
    reading = f" _`Lecturas:`{format_2}_"+"--".join(z)
    c = b + reading + " <br><br>"
    # Contruccion de string para visual estudio code
    for a,b in dicc.items():
        c = c + f"[{a}]" + b 

    c=c+"<br><br>"



    Lab_url = df_[df_.Sesion == Nsesion].Tipo_lab.values[0]
    Hw_url = df_[df_.Sesion == Nsesion].Tipo_hw.values[0]
    Project_url = df_[df_.Sesion == Nsesion].Tipo_project.values[0]
    Tipo_exam_url = df_[df_.Sesion == Nsesion].Tipo_exam.values[0]


    L = df_[df_.Sesion == Nsesion].Lab.values[0] 


    H = df_[df_.Sesion == Nsesion].Hw.values[0] 
    P = df_[df_.Sesion == Nsesion].Project.values[0]  
    W = df_[df_.Sesion == Nsesion].Exam.values[0]
    clave = [L, H, P, W]

    name_lab = df_[df_.Sesion == Nsesion].Name_lab_es.values[0]
    name_proj = df_[df_.Sesion == Nsesion].Name_project_es.values[0]
    names=[name_lab, None, name_proj, None]


    f_actual = Sesion_day[Sesion_day.Sesion == Nsesion].Fecha+timedelta(days=7)

    dicc = dicc_lab_hw_project_urls(Lab_url, Hw_url, Project_url,Tipo_exam_url, clave, names, f_actual)



    # Contruccion de string para visual estudio code
    for a,b in dicc.items():
        c = c + f"[{a}]" + b + "<br><br>"
        
    c=c+"|"
    
    return c

In [16]:
index=[]
for i in range(1,10):
    if(i==1):
        head = (f"""<div class=\"code-example\" markdown=\"1\">
    """)
    else:    
        
        head=""
    var = get_html_week(df_,Sesion_day,head, Nsesion = i)
    index.append(var)


In [18]:
index

['<div class="code-example" markdown="1">\n    \n|  Fecha   | Sesion          |\n|:---------|:--------------- |\n| Feb 07     | `1`{: .fw-700 .fs-5} `Introducción`{: .fw-700 .fs-5} <br><br> _`Lecturas:`{: .fw-700 .fs-3}_[1](https://inferentialthinking.com/chapters/01/what-is-data-science.html)--[1.1](https://inferentialthinking.com/chapters/01/1/intro.html)--[1.2](https://inferentialthinking.com/chapters/01/2/why-data-science.html)--[1.3](https://inferentialthinking.com/chapters/01/3/Plotting_the_Classics.html)--[1.3.1](https://inferentialthinking.com/chapters/01/3/1/Literary_Characters.html)--[1.3.2](https://inferentialthinking.com/chapters/01/3/2/Another_Kind_Of_Character.html) <br><br>[Presentacion](https://inferentialthinking.com/chapters/01/what-is-data-science.html){: .btn .float-left} [Demostracion](https://inferentialthinking.com/chapters/01/what-is-data-science.html){: .btn .float-left} [Video](https://inferentialthinking.com/chapters/01/what-is-data-science.html){: .btn .floa

In [21]:
Sesion_day

,Sesion,Fecha,Mes_dia,Semana
0,1,2024-02-07,Feb 07,1
1,2,2024-02-09,Feb 09,1
2,3,2024-02-12,Feb 12,2
3,4,2024-02-14,Feb 14,2
4,5,2024-02-16,Feb 16,2
5,6,2024-02-19,Feb 19,3
6,7,2024-02-21,Feb 21,3
7,8,2024-02-23,Feb 23,3
8,9,2024-02-26,Feb 26,4
9,10,2024-02-28,Feb 28,4


In [20]:
var = get_html_week(df_,Sesion_day,head, Nsesion = i)


In [18]:
step = 3
u = list(map(lambda x:"".join(index[x:x+step]), range(0, len(index), step)))

In [19]:
hh="\n\n".join(u)

In [20]:
hh="".join(hh)
hh = hh+"""
    
""" +  "<div>"+"""
    
    
    """

In [21]:
archivo = open("ejemplo.txt", "w")
archivo.write(hh)
archivo.close()
# Cierra el archivo